In [1]:
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd

import warnings

import time

import random

In [2]:
def ScanPageForAnime(starting_index):
    # base page showing 50 anime listings:
    result = requests.get("https://myanimelist.net/topanime.php?limit={}".format(starting_index))

    # page HTML content:
    src = result.content

    # parse src using BeautifulSoup:
    soup = BeautifulSoup(src, 'lxml')

    anime_block_list = soup.find_all("td",attrs={"class","title al va-t word-break"})

    anime_list = []
    for anime_block in anime_block_list:
        anime_link = anime_block.find("a").attrs["href"]
        anime_id = anime_link.replace("https://myanimelist.net/anime/", "").split("/")[0]
        anime_list.append(anime_id)
        
    return anime_list

In [3]:
def ScanAnimeDetails(mal_index):
    anime_link = "https://myanimelist.net/anime/{}".format(mal_index)

    result = requests.get(anime_link)
    src = result.content
    soup = BeautifulSoup(src, 'lxml')

    anime_title = "N\A"
    try:
        anime_title = soup.find("h1",attrs={"class","title-name h1_bold_none"}).find("strong").string
    except:
        print("Anime title error {}".format(mal_index))

    anime_score = "N\A"
    divs = soup.find_all("div",attrs={"class","fl-l score"})

    for div in divs:
        if div.find("div").text.replace('.', '', 1).isdigit():
            anime_score = div.find("div").text

    details = soup.find_all("div",attrs={"class","spaceit_pad"})

    anime_type = "N\A"
    anime_duration = "N\A"
    anime_studios = "N\A"
    anime_source = "N\A"
    anime_genres = "N\A"
    anime_themes = "N\A"
    anime_rating = "N\A"
    anime_popularity = "N\A"

    for div in details:
        if div.find("span") == None:
            continue
        if (div.find("span").string == "Type:"):
            try:
                anime_type = div.find("a").string
            except:
                print("Anime type error {}".format(mal_index))
        if (div.find("span").string == "Studios:"):
            try:
                studios = []
                for studio in div.find_all("a"):
                    studios.append(studio.string)
                anime_studios = studios
            except:
                print("Anime studio error {}".format(mal_index))
        if (div.find("span").string == "Source:"):
            anime_source = div.text.split(" ")[2].split("\n")[0]
        if (div.find("span").string == "Genres:"):
            genres = []
            for genre in div.find_all("a"):
                genres.append(genre.string)
            anime_genres = genres
        if (div.find("span").string == "Themes:"):
            themes = []
            for theme in div.find_all("a"):
                themes.append(theme.string)
            anime_themes = themes
        if (div.find("span").string == "Duration:"):
            txt = div.text.split(" ")[2:]
            anime_duration = " ".join(txt).split("\n")[0]
        if (div.find("span").string == "Rating:"):
            anime_rating = div.string
            if anime_rating == None:
                anime_rating = "N\A"
        if (div.find("span").string == "Popularity:"):
            anime_popularity = div.text.split(" ")[2].split("\n")[0][1:]
        
    return [anime_title, mal_index, anime_type, anime_duration, anime_studios, anime_source, anime_genres, anime_themes, anime_rating, anime_popularity, anime_score]

In [4]:
warnings.filterwarnings('ignore')

In [5]:
df = pd.DataFrame(columns=("Title", "MAL Id", "Type", "Episode Duration", "Studios", "Source Type", "Genres", "Themes", "Rating", "Popularity", "Score"))

In [6]:
for i in range(0, 200):
    ani = ScanPageForAnime(i*50)
    for anime_id in ani:
        data = ScanAnimeDetails(anime_id)
        a_series = pd.Series(data, index = df.columns)
        df = df.append(a_series, ignore_index=True)
        rand = random.uniform(1, 3)
        time.sleep(rand)

Anime type error 48653
Anime type error 42984
Anime type error 39112
Anime type error 34240
Anime type error 731
Anime type error 42076
Anime type error 39164
Anime type error 17949
Anime type error 42344
Anime type error 9930
Anime type error 47402
Anime type error 41195
Anime type error 41164
Anime type error 48876
Anime type error 49838
Anime type error 40569
Anime type error 36125
Anime type error 36494
Anime type error 42889
Anime type error 41132
Anime type error 30913
Anime type error 35006
Anime type error 39114
Anime type error 37370
Anime type error 39989
Anime type error 39065
Anime type error 39452
Anime type error 37954
Anime type error 1047
Anime type error 2768
Anime type error 32953
Anime type error 37266
Anime type error 32233
Anime type error 40828
Anime type error 39165
Anime type error 39875
Anime type error 39113
Anime type error 48554
Anime type error 6399
Anime type error 10445
Anime type error 34318
Anime type error 14951
Anime type error 34611
Anime type error 

In [8]:
filename = "anime_df.csv"

df.to_csv(filename, sep=';')

src=open(filename,"r")
fline="sep=;\n"
oline=src.readlines()
oline.insert(0,fline)
src.close()

src=open(filename,"w")
src.writelines(oline)
src.close()

UnicodeDecodeError: 'charmap' codec can't decode byte 0x90 in position 8: character maps to <undefined>